<h2>Steps to be Followed:</h2>
<ol>
1. importing necessary libraries</ol>
<ol>2. Creating S3 bucket</ol>
<ol>3. Mapping Test and Train data in S3 Bucket</ol>
<ol>4. Mapping the path of the model in S3</ol>


In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [2]:
bucket_name = "hmlproject"  #<-----Can Change this name as per variable name
my_region = boto3.session.Session().region_name #set the region of the instance
print(my_region)

us-east-2


In [3]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  hmlproject


In [4]:
import logging
import boto3
from botocore.exceptions import ClientError


def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True 

In [5]:
create_bucket(bucket_name, region="us-east-2")

ERROR:root:An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


False

In [6]:
# Setting An Output Path

prefix = "xgboost_as_a_built_in_algo"

output_path = 's3://{}/{}/output'.format(bucket_name,prefix)

print(output_path)

s3://hmlproject/xgboost_as_a_built_in_algo/output


<h3>Downloading the Dataset and storing in S3 bucket </h3>

In [7]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve('https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv', 'bank_clean.csv')
    print("Success :download bank_clean.csv")

except Exception as e:
    print("Data Load Error ", e)

Success :download bank_clean.csv


In [8]:
try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success data load into DataFrame')
except Exception as e:
    print('Failed to load data in DataFrame :', e)

Success data load into DataFrame


In [9]:
model_data

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,1,999,0,1,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
41184,46,1,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
41185,56,2,999,0,1,1,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
41186,44,1,999,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1


In [10]:
#Train Test Split using numpy

import numpy as np

train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7*(len(model_data)))])

<h1>Saving the Train and Test Split Data Frame</h1>

In [12]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/','train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [13]:
pd.concat([test_data['y_yes'], test_data.drop(['y_yes', 'y_no'],axis=1)],axis=1).to_csv('test.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/','test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

<h1>Building Model - XGBoost - Inbuilt Algorithm</h1>

In [28]:
##This line will look for XGBoost Image Url and build the XGBoost Container.
##Specify the repo version depend upon your preference

container = sagemaker.image_uris.retrieve(boto3.Session().region_name,
                          'xgboost', 
                          container_version='1.2-2')

FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/image_uri_config/us-east-2.json'

In [39]:
#Initializing the Hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round" :"10"}

In [40]:
#Construct the Sagemaker Estimator that call the XGBoost-container

estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          train_max_run = 300,
                                          train_max_wait = 600,)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [41]:
# execute the XGBoost training job
estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

2021-10-05 14:10:22 Starting - Starting the training job...
2021-10-05 14:10:45 Starting - Launching requested ML instancesProfilerReport-1633443022: InProgress
......
2021-10-05 14:11:46 Starting - Preparing the instances for training......
2021-10-05 14:12:50 Downloading - Downloading input data...
2021-10-05 14:13:06 Training - Downloading the training image...
2021-10-05 14:13:46 Uploading - Uploading generated training model[2021-10-05 14:13:41.844 ip-10-0-175-247.us-east-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-05:14:13:41:INFO] Imported framework sagemaker_xgboost_container.training
[2021-10-05:14:13:41:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2021-10-05:14:13:41:INFO] No GPUs detected (normal if no gpus installed)
[2021-10-05:14:13:41:INFO] Running XGBoost Sagemaker in algorithm mode
[2021-10-05:14:13:41:INFO] Determined delimiter of CSV input is ','
[2021-10-05:14:13:41:

In [45]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type="ml.t2.medium")

--------!

<h3>Prediction of the Test Data
</h3>

In [48]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
# xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


In [49]:
predictions_array

array([0.10390978, 0.10390978, 0.10390978, ..., 0.09552468, 0.09552468,
       0.09552468])

In [50]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10823)    30% (113)
Purchase        10% (1159)     70% (262) 



<h3>Deleting the Endpoints</h3>

In [51]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[{'ResponseMetadata': {'RequestId': '54HCJ9FD831CDBV0',
   'HostId': '/WxnsuLPT6+DgaXEPXmrmx8NcasQ/c2939IsiGuoo5s+wTueY6M0tCj6MhRrJEayFReIpMxuMcs=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '/WxnsuLPT6+DgaXEPXmrmx8NcasQ/c2939IsiGuoo5s+wTueY6M0tCj6MhRrJEayFReIpMxuMcs=',
    'x-amz-request-id': '54HCJ9FD831CDBV0',
    'date': 'Tue, 05 Oct 2021 14:38:53 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost_as_a_built_in_algo/output/sagemaker-xgboost-2021-10-05-14-10-22-423/rule-output/ProfilerReport-1633443022/profiler-output/profiler-reports/OverallSystemUsage.json'},
   {'Key': 'xgboost_as_a_built_in_algo/output/sagemaker-xgboost-2021-10-05-14-03-06-197/profiler-output/system/incremental/2021100514/1633442760.algo-1.json'},
   {'Key': 'xgboost_as_a_built_in_algo/output/sagemaker-xgboost-2021-10-05-14-10-22-423/output/model.tar.gz'},
  